# Five Clique
This notebook solves a problem from one of Matt Parker's YouTube videos: https://www.youtube.com/watch?v=_-AfhLQfb6w

The problem is to find 5 words with 5 letters each, and all letters have to be unique.

I've implemented some of the fast methods mentioned near the end of the video to achieve a runtime of less than one minute in Python.

This solution finds all possible combinations of words at the cost of being sub-optimal, but it's still pretty fast!
<span style="font-size:1.44em;line-height:0">🏎️</span>

#### **Configuration:**

##### **Select which word dataset to use:**

0. words_alpha.txt from https://github.com/dwyl/english-words
1. unigram_freq.csv from https://www.kaggle.com/datasets/rtatman/english-word-frequency

In [1]:
dataset = 0

#### Setup:

##### **Imports:**

In [2]:
from time import time
from math import floor
from collections import Counter
from IPython.display import clear_output

##### **Start a timer.**

In [3]:
class Timer:
    def __init__(self):
        self.start = time()
    
    def __str__(self):
        seconds = time() - self.start
        if seconds < 10:
            return 'Time: {:d} ms'.format(floor(seconds * 1000))
        seconds = floor(seconds)
        if seconds < 60:
            return 'Time: {:d} s'.format(seconds)
        minutes, seconds = divmod(seconds, 60)
        if minutes < 60:
            return 'Time: {:d}:{:0>2d}'.format(minutes, seconds)
        hours, minutes = divmod(minutes, 60)
        return 'Time: {:d}:{:0>2d}:{:0>2d}'.format(hours, minutes, seconds)

timer = Timer()

##### **Load 5-letter words with unique letters.**

In [4]:
match dataset:
    case 0:
        print('Using dataset words_alpha.txt')
        with open('words_alpha.txt') as file:
            words = file.read().split()
    case 1:
        print('Using dataset unigram_freq.csv')
        with open('unigram_freq.csv') as file:
            words = [item.split(',')[0] for item in file.read().split()]
    case _:
        raise ValueError('Invalid dataset')
print('Unfiltered:', len(words))

words = [word.upper() for word in words if len(word) == 5]
print('5-letters:', len(words))

words = [word for word in words if len(set(word)) == 5]
print('Unique letters:', len(words))

if dataset == 1:
    print('Using only the 5000 most common words based on frequency data')
    words = words[:5000]

print(timer)

Using dataset words_alpha.txt
Unfiltered: 370105
5-letters: 15920
Unique letters: 10175
Time: 100 ms


##### **Construct an alphabet with the lowest frequency characters first.**

In [5]:
freqs = Counter(char for word in words for char in word)
alphabet = ''.join([char for char, freq in reversed(freqs.most_common())])
print('Alphabet:', alphabet)
print(timer)

Alphabet: QJXZVFWKBGPMHYDCUNTLOIRSEA
Time: 117 ms


##### **Function to get the position of the first character in the alphabet from a code:**

In [6]:
def first_char(code):
    return (code & -code).bit_length() - 1

##### **Convert the words to order-independent binary codes based on the alphabet.**

In [7]:
codes = [[] for _ in range(22)]
for i, word in enumerate(words):
    code = sum([1 << alphabet.index(char) for char in word])
    first = first_char(code)
    codes[first].append((code, i))
print(timer)

Time: 169 ms


##### **Functions to get codes for the next word:**

In [8]:
def next_codes(code):
    global codes
    first_missing = first_char(~code)
    return filter(lambda item: code & item[0] == 0, codes[first_missing])

def skip_codes(code):
    global codes
    first_missing = first_char(~code)
    second_missing = first_char(~(code + (1 << first_missing)))
    return filter(lambda item: code & item[0] == 0, codes[second_missing])

##### **Function to add all possible next words for all sequences:**

In [9]:
def add_word(prev, name):
    global timer
    progress = 0
    last_update = time()
    new = []
    for code1, skip, *i1s in prev:
        for code2, i2 in skip_codes(code1):
            new.append((code1 + code2, True, *i1s, i2))
        if not skip:
            for code2, i2 in next_codes(code1):
                new.append((code1 + code2, False, *i1s, i2))
        progress += 1
        if time() - last_update > 0.1:
            last_update += 0.1
            clear_output(wait=True)
            print('Progress: {} / {} ({:.1%})'.format(progress, len(prev), progress / len(prev)))
            print('{}:'.format(name), len(new))
            print(timer)
    clear_output(wait=True)
    print('{}:'.format(name), len(new))
    print(timer)
    return new

#### Processing:

##### **Build every possible sequence.**

In [10]:
ones = add_word([(0, False)], 'Ones')

Ones: 361
Time: 205 ms


In [11]:
twos = add_word(ones, 'Twos')

Twos: 26241
Time: 278 ms


In [12]:
threes = add_word(twos, 'Threes')

Threes: 174458
Time: 1560 ms


In [13]:
fours = add_word(threes, 'Fours')

Fours: 74033
Time: 17 s


In [14]:
fives = [' '.join(words[i] for i in idxs) for _, _, *idxs in add_word(fours, 'Fives')]

Fives: 831
Time: 27 s


#### **Output:**

##### **Display the first 100 sequences.**

In [15]:
for five in fives[:100]:
    print(five)

BEJAN FLDXT GROSZ VICKY WHUMP
BEJIG FLDXT KNYAZ SUPVR MOWCH
BEJIG FLDXT KNYAZ VROWS CHUMP
BENJY FLDXT GIZMO SUPVR CHAWK
BENJY FLDXT GIZMO SUPVR WHACK
BENJY FLDXT GROSZ AVICK WHUMP
BENJY FLDXT OGHUZ VAMPS WRICK
BENJY FLDXT VIZOR GAWKS CHUMP
BENJY FLDXT VIZOR WHAMP GUCKS
BRUJA FLDXT ZYGON CHIVW KEMPS
BRUJA FLDXT ZYGON CHIVW SKEMP
FJELD AMPYX BORTZ CHIVW GUNKS
FJORD EXPWY KLUTZ VANGS CHIMB
FJORD PBXES KLUTZ CHIVW MANGY
FJORD PBXES MUNTZ CHIVW GLAKY
FJORD VIBEX MUNTZ SWACK GLYPH
FJORD VIBEX MUNTZ WACKS GLYPH
FJORD VIBEX WALTZ GUCKS NYMPH
JACKY FLDXT GINZO VERBS WHUMP
JACKY FLDXT GROSZ VINEW BUMPH
JACKY FLDXT ZINGS BEVOR WHUMP
JACKY FLDXT ZINGS VOWER BUMPH
JACKY PBXES FULTZ MORDV WHING
JACKY PBXES ZHMUD VINGT FROWL
JACKO FLDXT ZINGY VERBS WHUMP
JACKO FLDXT ZINGS VERBY WHUMP
JACKO FLDXT ZINGS WYVER BUMPH
JACKS FLDXT BIZEN GROVY WHUMP
JACKS FLDXT BUREZ VYING WHOMP
JACKS FLDXT GINZO VERBY WHUMP
JACKS FLDXT GINZO WYVER BUMPH
JACKS FLDXT WINZE GROVY BUMPH
JACKS FLDXT WIZEN GROVY BUMPH
JACKS FLDX